In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=b5c72fc02dccd20a9607aca1e5eb9977530c4d741cad0d8bf1aa11c648199cf4
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
spark = SparkSession.builder.appName('Food DF').getOrCreate()
pyspark.sql.session.SparkSession

pyspark.sql.session.SparkSession

In [2]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

In [3]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [6]:
data = spark.read.csv("drive/My Drive/Colab Notebooks/Data/Weather Data in India from 1901 to 2017.csv", header=True,inferSchema=True)
data = data.select([col for col in data.columns if col != '_c0'])
data.show()
data.printSchema()

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|
|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|
|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|
|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|
|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|
|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|
|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|
|1908|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|
|1909|17.79|19.05| 23.4|25.76|27.97|27.67|26.56|26.43|25.47|24.37|22.01|18.86|
|1910|18.14|19.72| 22.9|25.96|28.36|27.72|26.93|26.6

In [27]:
#1. Add column average temperature for each year
avg_temp = data.withColumn('Average',(col('JAN')+col('FEB')+col('MAR')+col('APR')+col('MAY')+col('JUN')+col('JUL')+col('AUG')+col('SEP')+col('OCT')+col('NOV')+col('DEC'))/12)
avg_temp.show()

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+
|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|           Average|
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+
|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|24.224166666666665|
|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|24.326666666666664|
|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|23.980833333333333|
|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|23.849999999999998|
|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|           23.9325|
|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|24.099999999999998|
|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|24.007499999999997|
|1908|18.35|19.73|22

In [26]:
#2. Add row average temperature of each month
avg_temp1 = data.select([cols for cols in data.columns if cols != 'YEAR'])
avg_dic = {}
for i in avg_temp.columns:
  avg_dic[i] = data.select(i).agg({i:'avg'}).collect()[0][0]
avg_dic

{'JAN': 18.42324786324786,
 'FEB': 20.137948717948717,
 'MAR': 23.434188034188036,
 'APR': 26.514102564102565,
 'MAY': 28.386410256410255,
 'JUN': 28.300940170940162,
 'JUL': 27.369230769230775,
 'AUG': 26.940085470085464,
 'SEP': 26.342649572649577,
 'OCT': 24.742051282051282,
 'NOV': 21.765726495726508,
 'DEC': 19.173333333333332}

In [13]:
new_row = Row(_c0='Average', YEAR='----', JAN=avg_dic['JAN'], FEB=avg_dic['FEB'], MAR=avg_dic['MAR'], APR=avg_dic['APR'],
                                          MAY=avg_dic['MAY'], JUN=avg_dic['JUN'], JUL=avg_dic['JUL'], AUG=avg_dic['AUG'],
                                          SEP=avg_dic['SEP'], OCT=avg_dic['OCT'], NOV=avg_dic['NOV'], DEC=avg_dic['DEC']
             )
new_row_data = spark.createDataFrame([new_row])
data_new = spark.read.csv("drive/My Drive/Colab Notebooks/Data/Weather Data in India from 1901 to 2017.csv", header=True,inferSchema=True)
data_new = data_new.union(new_row_data)
spark.createDataFrame(data_new.tail(10)).show()

+-------+----+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|    _c0|YEAR|              JAN|               FEB|               MAR|               APR|               MAY|               JUN|               JUL|               AUG|               SEP|               OCT|               NOV|               DEC|
+-------+----+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|    108|2009|            19.79|             21.66|             24.55|             27.35|             28.71|             28.77|             27.83|             27.85|             27.11|              25.2|             22.31|             20.22|
|    109|2010|            19.15|

In [14]:
#3. Add columns Min and Max for each year with corresponding values
data3 = data.withColumn('Min',least(col('JAN'),col('FEB'),col('MAR'),col('APR'),col('MAY'),col('JUN'),col('JUL'),col('AUG'),col('SEP'),col('OCT'),col('NOV'),col('DEC')))\
         .withColumn('Max',greatest(col('JAN'),col('FEB'),col('MAR'),col('APR'),col('MAY'),col('JUN'),col('JUL'),col('AUG'),col('SEP'),col('OCT'),col('NOV'),col('DEC')))
data3.show()

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|  Min|  Max|
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|17.99| 28.6|
|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|18.78|28.68|
|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|18.29|28.41|
|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|17.77|27.85|
|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71| 17.4|28.69|
|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55| 17.5|29.06|
|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|18.46|27.66|
|1908|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|18.15|28.49|

In [15]:
#4. Create dataframe called 'decade', storing average temperatures of each month in the respective decade. Decade 1910 is from 1901 to 1910

decade = data.groupBy(floor(col("YEAR") / 10) * 10).agg(avg("JAN").alias('JAN'),avg("FEB").alias('FEB'),avg("MAR").alias('MAR'),avg("APR").alias('APR'),avg("MAY").alias('MAY'),
    avg("JUN").alias('JUN'),avg("JUL").alias('JUL'),avg("AUG").alias('AUG'),avg("SEP").alias('SEP'),avg("OCT").alias('OCT'),avg("NOV").alias('NOV'),avg("DEC").alias('DEC'))

decade.withColumnRenamed('(floor((YEAR / 10)) * 10)','YEAR').orderBy('YEAR').show()

+----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|YEAR|               JAN|               FEB|               MAR|               APR|               MAY|               JUN|               JUL|               AUG|               SEP|               OCT|               NOV|               DEC|
+----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|1900|18.154444444444444| 19.34777777777778| 22.81666666666667|             26.32|28.184444444444445|28.203333333333337|             27.31|26.766666666666666| 26.04222222222222|24.736666666666668|21.585555555555555| 18.73222222222222|
|1910|            18.306|            19.673|            22.7

In [31]:
#5. Which year is the hottest year? (Based on avg temp)
avg_temp.select('YEAR').where(col('Average')==avg_temp.select('Average').agg({'Average':'max'}).collect()[0][0]).show()

+----+
|YEAR|
+----+
|2016|
+----+



In [28]:
#6. Which year is the coldest year? (Based on avg temp)
avg_temp.select('YEAR').where(col('Average')==avg_temp.select('Average').agg({'Average':'min'}).collect()[0][0]).show()

+----+
|YEAR|
+----+
|1917|
+----+



In [29]:
#7. Find out which years recorded minimum and maximum temperatures. (Not based on avg temp)
data3.select('YEAR').where(col('Min')==data3.select('Min').agg({'Min':'min'}).collect()[0][0]).show()
data3.select('YEAR').where(col('Max')==data3.select('Max').agg({'Max':'max'}).collect()[0][0]).show()

+----+
|YEAR|
+----+
|1918|
+----+

+----+
|YEAR|
+----+
|1921|
+----+



In [30]:
#8. How much raise is observed in min and max temperature for each month?
data4 = data.select([cols for cols in data.columns if cols != 'YEAR'])
for i in data4.columns:
  print(f"{i}:{data4.select(i).agg({i:'max'}).collect()[0][0]-data.select(i).agg({i:'min'}).collect()[0][0]}")

JAN:3.6700000000000017
FEB:5.789999999999999
MAR:4.829999999999998
APR:4.719999999999999
MAY:3.8100000000000023
JUN:2.5500000000000007
JUL:1.9899999999999984
AUG:1.9600000000000009
SEP:2.6400000000000006
OCT:3.719999999999999
NOV:3.330000000000002
DEC:3.91
